In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import Parsers as P

# Tutorial of the PwParser class

This tutorial describes the usage of the class PwParser of mppi used to extract information for the XML output file data-file-schema produced
by pw.

## Parse of a output file of Silicon 

__Follow the tutorial for the QeCalculator to produce the xml file used by the PwParser.__

The class is initialized by specifying the name of the xml file including its relative path 

In [34]:
results = P.PwParser('QeCalculator_test/ecut_40.save/data-file-schema.xml')

Parse file : QeCalculator_test/ecut_40.save/data-file-schema.xml


When the object is initialized several attributes are set, for instance

In [35]:
results.units

'Hartree atomic units'

In [36]:
print(results.natoms)
print(results.atomic_species)
print(results.atomic_positions)
print(results.nbands)
print(results.nkpoints)
print(results.spin_degen)

2
{'Si': ['2.808600000000000e1', 'Si.pbe-mt_fhi.UPF']}
[['Si', [-1.2875, 1.2875, 1.2875]], ['Si', [1.2875, -1.2875, -1.2875]]]
4
32
2


We can print the ks energies and weight for each kpoint

In [37]:
for k,e,w in zip(results.kpoints,results.evals,results.weights):
    print(k,e,w)

[0.0, 0.0, 0.0] [-0.21237912  0.22480614  0.2248063   0.22480631] [0.00925926]
[-0.1666666666666667, 0.1666666666666667, -0.1666666666666667] [-0.19919081  0.1375121   0.20844019  0.20844032] [0.05555556]
[-0.3333333333333333, 0.3333333333333333, -0.3333333333333333] [-0.16220055  0.02987321  0.18835681  0.18835693] [0.05555556]
[0.5, -0.5, 0.5] [-0.12757073 -0.02957081  0.18109866  0.18109877] [0.02777778]
[0.0, 0.3333333333333333, 0.0] [-0.1947076   0.14932703  0.18151791  0.18151797] [0.05555556]
[-0.1666666666666667, 0.5, -0.1666666666666667] [-0.16499775  0.06224759  0.15213905  0.16134207] [0.11111111]
[0.6666666666666666, -0.3333333333333334, 0.6666666666666666] [-0.12170123 -0.01655792  0.12356531  0.16017936] [0.11111111]
[0.5, -0.1666666666666667, 0.5] [-0.13567106  0.00519559  0.11383639  0.17756933] [0.11111111]
[0.3333333333333333, 2.775557561562891e-17, 0.3333333333333333] [-0.17775365  0.08840302  0.13722415  0.2041809 ] [0.05555556]
[0.0, 0.6666666666666666, 0.0] [-0.14

There are also several get methods, for instance

In [41]:
results.get_fermi()

Fermi energy attribute not found in the ouput file. Maybe `fixed` occupation type is used?


The method get_evals  return an array with the ks energies for each kpoint. The energies are expressed in eV and the 
energy of VBM is used as reference. A gap, both direct or indirect, can be set. In this case the energies of the empty ks states is
shifted. This procedure __does not__ update the occupation levels of the empty states

In [42]:
results.get_evals(set_gap=1.2)[0]

There are no empty bannds. `Set gap` has not been applied


array([-1.18964214e+01, -4.48659002e-06, -1.24152244e-09,  0.00000000e+00])

In [43]:
results.get_gap()

There are no empty states. Gap cannot be computed.


We test the PwParser using a nscf output file, in this case empty bands are present and the gap can be computed

In [44]:
result_nscf = P.PwParser('QeCalculator_test/bands_12.save/data-file-schema.xml')

Parse file : QeCalculator_test/bands_12.save/data-file-schema.xml


In [45]:
result_nscf.occupations[0]

array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [46]:
result_nscf.get_gap()

Indirect gap system
Gap : 0.7363567574170765 eV
Direct gap : 2.5651333558109863 eV


{'gap': 0.7363567574170765,
 'direct_gap': 2.5651333558109863,
 'position_cbm': 12,
 'positon_vbm': 0}

In [47]:
result_nscf.get_evals(set_direct_gap=3.0)[0]

Apply a scissor of 0.43486664418901366 eV


array([-1.18964110e+01, -9.74240383e-07, -9.74234497e-07,  0.00000000e+00,
        3.00000000e+00,  3.00000056e+00,  3.00000056e+00,  3.58353139e+00,
        8.17161688e+00,  8.17161688e+00,  8.29564293e+00,  1.16527216e+01])

we see that in this case the energy of the 5-th states has been shift to 3 eV to implement the set_direct_gap requirement. 

## Parse of a Graphene output file

__Run the Analysis_BandStructure notebook to produce the xml file used by the PwParser.__

We parse a graphene output file to test if systems without a gap are correctly parsed.

In [48]:
results_gra = P.PwParser('Pw_bands/graphene_nscf.save/data-file-schema.xml')

Parse file : Pw_bands/graphene_nscf.save/data-file-schema.xml


In [49]:
results_gra.occupations_kind

'smearing'

In [50]:
results_gra.occupations[0]

array([1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       4.22181265e-29, 6.89778699e-37, 1.76721673e-41, 1.28104982e-57])

In [51]:
results_gra.occupations[18] #the position of K

array([1. , 1. , 1. , 0.5, 0.5, 0. , 0. , 0. ])

In [52]:
results_gra.get_gap()

Direct gap system
Gap : 2.715685792485934e-10 eV


{'gap': 2.715685792485934e-10,
 'direct_gap': 2.715685792485934e-10,
 'position_cbm': 18,
 'positon_vbm': 18}

In [53]:
results_gra.get_fermi()

-1.734315271671807

In [12]:
import xml.etree.ElementTree as ET
data = ET.parse('Pw_bands/graphene_nscf.save/data-file-schema.xml').getroot()

In [25]:
data.find('output/band_structure/fermi_energy').text

'-6.373491124898257e-2'